<a href="https://colab.research.google.com/github/DJCordhose/deep-learning-crash-course-notebooks/blob/master/tf-v2/tf-low-level.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Introduction to Neural Networks with TensorFlow and Keras layers

## Converted to TensorFlow 2


In [0]:
import warnings
warnings.filterwarnings('ignore')

In [2]:
%matplotlib inline
%pylab inline
import matplotlib.pyplot as plt

Populating the interactive namespace from numpy and matplotlib


In [3]:
import pandas as pd
print(pd.__version__)

0.22.0


In [0]:
!pip install tf-nightly-gpu-2.0-preview >/dev/null

In [5]:
import tensorflow as tf
print(tf.__version__)

2.0.0-dev20190306


In [6]:
# a small sanity check, does tf seem to work ok?
hello = tf.constant('Hello TF!')
print("This works: {}".format(hello))

This works: b'Hello TF!'


In [7]:
# this should return True even on Colab
tf.test.is_gpu_available()

True

## Loading and preparing our data set for classification

In [8]:
!curl -O https://raw.githubusercontent.com/DJCordhose/deep-learning-crash-course-notebooks/master/data/insurance-customers-1500.csv

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 26783  100 26783    0     0  97392      0 --:--:-- --:--:-- --:--:-- 97392


In [0]:
df = pd.read_csv('./insurance-customers-1500.csv', sep=';')

In [10]:
df.head()

,speed,age,miles,group
0,98.0,44.0,25.0,1
1,118.0,54.0,24.0,1
2,111.0,26.0,34.0,0
3,97.0,25.0,10.0,2
4,114.0,38.0,22.0,1


In [11]:
df.describe()

,speed,age,miles,group
count,1500.000000,1500.000000,1500.000000,1500.000000
mean,122.492667,44.980667,30.434000,0.998667
std,17.604333,17.130400,15.250815,0.816768
min,68.000000,16.000000,1.000000,0.000000
25%,108.000000,32.000000,18.000000,0.000000
50%,120.000000,42.000000,29.000000,1.000000
75%,137.000000,55.000000,42.000000,2.000000
max,166.000000,100.000000,84.000000,2.000000


## First important concept: You train a machine with your data to make it learn the relationship between some input data and a certain label - this is called supervised learning

<img src='https://raw.githubusercontent.com/DJCordhose/deep-learning-crash-course-notebooks/master/img/encoding3.jpg'>

In [0]:
# we deliberately decide "group" is going to be our label, 
# it is often named lower case y
y = df['group']

In [0]:
# since 'group' is now the label we want to predict, 
# we need to remove it from the training data 
df.drop('group', axis='columns', inplace=True)

In [0]:
# input data often is named upper case X, 
# the upper case indicates, that each row is a vector
X = df.as_matrix()

## Neural Networks using TensorFlow and Keras layers
* Neural Networks consist of artificial neurons you organize in layers
* each neuron is very simple, but, theoretically, having enough of them in a single layer can approximate any funtion
* practically, we use 2 or 3 layers, as this has turned out to work well
* the more neurons and the more layers you use the longer the network takes to train
* neural networks often are no longer approachable using cross validation and grid search to find suitable hyper parameters

## Neuron (aka node or unit)

A neuron takes a number of numerical inputs, multiplies each with a weight, sums up all weighted input and adds bias (constant) to that sum. From this it creates a single numerical output. For one input (one dimension) this would be a description of a line. For more dimensions this describes a hyper plane that can serve as a decision boundary. Typically, this output is transformed using an activation function which compresses the output to a value between 0 and 1 (sigmoid), or between -1 and 1 (tanh) or sets all negative values to zero (relu).

It is not really important to understand the details of a neural network. Practically how you configure them to form something more powerful is much more important. This, however, is still a very experimental domain, so there really is no conscise explanation and understanding how they work.

<img src='https://raw.githubusercontent.com/DJCordhose/deep-learning-crash-course-notebooks/master/img/neuron.jpg'>

### We use a sequential mode, that means data flows without junctions from in to out

In [0]:
model = tf.keras.Sequential()

### We start with a single fully connected layer having 50 neurons
* we have three inputs
  * age 
  * speed
  * miles
* activation function is tanh
* why these parameters: random for now

In [0]:
from tensorflow.keras.layers import Dense

model.add(Dense(50, name='hidden1', activation='tanh', input_dim=3))

### The final layer just transforms to likelyhood for each of our 3 classes

In [0]:
num_categories = 3
model.add(Dense(num_categories, name='softmax', activation='softmax'))

### First, let us have a look at how the input and output from this model would look like

* this model has not been trainined, so do not expect the outputs to be reasonable
* we are only interested in the format of input and output
* note that there is a mismatch between prediction and our known truths in format
* we will fix this in the next step

In [0]:
input = X[0:10]

In [19]:
# combinations of customer data
input

array([[ 98.,  44.,  25.],
       [118.,  54.,  24.],
       [111.,  26.,  34.],
       [ 97.,  25.,  10.],
       [114.,  38.,  22.],
       [130.,  55.,  34.],
       [118.,  40.,  51.],
       [143.,  42.,  34.],
       [120.,  41.,  42.],
       [148.,  33.,  53.]])

In [20]:
# predicted output: likeliyhoods for groups
model.predict(input)

array([[0.40454787, 0.47451735, 0.12093481],
       [0.40387106, 0.47560373, 0.12052518],
       [0.38871348, 0.48780105, 0.12348553],
       [0.30377945, 0.54448265, 0.15173797],
       [0.29907176, 0.5965444 , 0.10438386],
       [0.40074256, 0.47522762, 0.12402984],
       [0.39941972, 0.47566476, 0.12491554],
       [0.3480127 , 0.5354465 , 0.11654083],
       [0.39925212, 0.47560307, 0.12514485],
       [0.4003986 , 0.47425112, 0.12535033]], dtype=float32)

In [21]:
# true, known output
y[0:10]

0    1
1    1
2    0
3    2
4    1
5    0
6    0
7    1
8    2
9    0
Name: group, dtype: int64

### These are the parameters of the model that need to be learned

In [22]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
hidden1 (Dense)              (None, 50)                200       
_________________________________________________________________
softmax (Dense)              (None, 3)                 153       
Total params: 353
Trainable params: 353
Non-trainable params: 0
_________________________________________________________________


### Bringing it all together
* _sparse_categorical_crossentropy_
  * _crossentropy_: Loss is defined by https://en.wikipedia.org/wiki/Cross_entropy
  * _categorical_: we are comparing categorical data
  * _sparse_: allows us to leave our labels as they are without explicitly turning them into a one-hot encoding 
* _adam_: is the least tedious algorithm to minimize loss (http://cs231n.github.io/neural-networks-3/#ada)
  * auto-tunes most important parameters including learning rate   

In [0]:
model.compile(loss='sparse_categorical_crossentropy',
             optimizer='adam')

# Caution: we have not trained our model, yet, the parameters are still initinialized randomly